<a href="https://colab.research.google.com/github/HazSyl1/LORA/blob/main/LLM_CHATBOT_MISTRAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-st6r4vkc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-st6r4vkc
  Resolved https://github.com/huggingface/transformers to commit 2cc8cf6ce7ae0416561acbb639df4bbc5f409b6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
from datasets import load_dataset , Dataset
from peft import LoraConfig,AutoPeftModelForCausalLM ,prepare_model_for_kbit_training , get_peft_model
from transformers import AutoModelForCausalLM ,AutoTokenizer , GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os


In [6]:
data=load_dataset("tatsu-lab/alpaca",split="train")
data_df=data.to_pandas()
data_df=data_df[:5000]
data_df["text"]=data_df[["input","instruction","output"]].apply(lambda x: "###Human: "+x["instruction"]+ " "+x["input"]+"###Assistant: " +x["output"] ,axis=1)
data=Dataset.from_pandas(data_df)


In [7]:
data['text'][0]

'###Human: Give three tips for staying healthy. ###Assistant: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [8]:
tokenizer=AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token=tokenizer.eos_token

quantization_config_loading=GPTQConfig(bits=4,disable_exllama=True,tokenizer=tokenizer)
model=AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
    quantization_config=quantization_config_loading,
    device_map="auto"
)

model.config.use_cache=False
model.config.pretrained_tp=1
model.gradient_checkpointing_enable()
model=prepare_model_for_kbit_training(model) #becomes prepared for 4 bit training



tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
peft_config=LoraConfig(
    r=16,lora_alpha=16,lora_dropout=0.05,bias="none",task_type="CAUSAL_LM",target_modules=["q_proj","v_proj"]
)
model=get_peft_model(model,peft_config)

training_arguments=TrainingArguments(
    output_dir="mistral-finetuned-alpaca",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100, #at 100 step it'll give the log of evaluation
    num_train_epochs=1,
    max_steps=250, #here we've 8*1*250= 2000 steps.Meaning it'll work for 2000 steps onlt
    fp16=True,
    push_to_hub=True
)

In [10]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False, #packing the tokens that dont fit in the max_token_length , in the second batch/iter
    max_seq_length=512
)
trainer.train()

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.454800
200,1.271600


TrainOutput(global_step=250, training_loss=1.336803451538086, metrics={'train_runtime': 1452.0918, 'train_samples_per_second': 1.377, 'train_steps_per_second': 0.172, 'total_flos': 310614365306880.0, 'train_loss': 1.336803451538086, 'epoch': 0.4})

In [13]:
! cp -r /content/mistral-finetuned-alpaca /content/drive/MyDrive/

In [12]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [14]:
model.save_pretrained("HazSylvia/Fine-Tuned-Mistral")

tokenizer.save_pretrained("HazSylvia/Fine-Tuned-Mistral")

('HazSylvia/Fine-Tuned-Mistral/tokenizer_config.json',
 'HazSylvia/Fine-Tuned-Mistral/special_tokens_map.json',
 'HazSylvia/Fine-Tuned-Mistral/tokenizer.model',
 'HazSylvia/Fine-Tuned-Mistral/added_tokens.json',
 'HazSylvia/Fine-Tuned-Mistral/tokenizer.json')